In [1]:
# Import necessary libraries
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import requests
import cv2

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load training data with augmentation
train_generator = datagen.flow_from_directory(
    'train',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)

# Load validation data
validation_generator = tf.keras.utils.image_dataset_from_directory(
    'valid',
    labels="inferred",
    label_mode="categorical",
    image_size=(128, 128),
    batch_size=32
)

# Transfer learning with VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
for layer in base_model.layers:
    layer.trainable = False

# Adding custom layers on top of VGG16
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
disease_prediction = Dense(38, activation='softmax', name='disease')(x)
severity_prediction = Dense(3, activation='softmax', name='severity')(x)  # Assuming 3 severity levels

model = Model(inputs=base_model.input, outputs=[disease_prediction, severity_prediction])

# Compile the model
model.compile(
    optimizer='adam',
    loss={'disease': 'categorical_crossentropy', 'severity': 'categorical_crossentropy'},
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

# Real-time disease detection with webcam
def predict_image(image):
    img = cv2.resize(image, (128, 128))
    img = np.expand_dims(img, axis=0)
    predictions = model.predict(img)
    return predictions

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    predictions = predict_image(frame)
    disease = np.argmax(predictions[0])
    severity = np.argmax(predictions[1])
    cv2.putText(frame, f'Disease: {disease}, Severity: {severity}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.imshow('Real-time Disease Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Weather data integration
api_key = '1b270cb5fbccb5332d86ec8f4e997a2e'
lat = lon = 'http://api.openweathermap.org/geo/1.0/direct?q={city_name}&appid={api_key}'
base_url = 'https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={api_key}'
city_name = 'hyderabad'
complete_url = base_url + 'appid=' + api_key + '&q=' + city_name
response = requests.get(complete_url)
weather_data = response.json()

def get_recommendation(disease, severity, weather_data):
    # Provide recommendations based on disease, severity, and weather data
    if disease == 'disease_name' and severity == 'severity_level':
        if weather_data['main']['temp'] > 30:
            return 'Recommendation for hot weather'
        else:
            return 'Recommendation for cold weather'
    # Add more logic for other diseases and weather conditions

recommendation = get_recommendation('disease_name', 'severity_level', weather_data)
print(recommendation)



Found 70295 images belonging to 38 classes.
Found 17572 files belonging to 38 classes.



Epoch 1/10




InvalidArgumentError: Graph execution error:

Detected at node categorical_crossentropy_1/softmax_cross_entropy_with_logits defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "D:\disk c\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "D:\disk c\anaconda3\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance

  File "D:\disk c\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 736, in start

  File "D:\disk c\anaconda3\Lib\site-packages\tornado\platform\asyncio.py", line 195, in start

  File "D:\disk c\anaconda3\Lib\asyncio\base_events.py", line 607, in run_forever

  File "D:\disk c\anaconda3\Lib\asyncio\base_events.py", line 1922, in _run_once

  File "D:\disk c\anaconda3\Lib\asyncio\events.py", line 80, in _run

  File "D:\disk c\anaconda3\Lib\site-packages\ipykernel\kernelbase.py", line 516, in dispatch_queue

  File "